# Sympy to compute CCSD equations

Let us try to make a sympy code to obtain automatic generation of 
CCSD equations.

__Problem with dummy index__

## first example
Using sympy.physics.secondquant

* symbols can be defined for hole and particles or dummy

* Fermion Creation operator is Fd

* Fermion Annihilation operator is F

* contraction by wicks

Fully contract 
$$a^\dagger_i a_a a^\dagger_p a_q a^\dagger_b a_j$$

In [11]:
from sympy import symbols, latex, WildFunction, collect, Rational, simplify, Dummy
from sympy.physics.secondquant import F, Fd, wicks, AntiSymmetricTensor, substitute_dummies, NO, evaluate_deltas

from IPython.display import display, Math

#---define symbols in secondquantization 
i, j = symbols('i,j', below_fermi=True)
a, b = symbols('a,b', above_fermi=True)
p, q = symbols('p,q',cls=Dummy)
#---creation and annihilation operators
display(Math(latex(Fd(i)*F(a)*Fd(p)*F(q)*Fd(b)*F(j))))
simplify(wicks(Fd(i)*F(a)*Fd(p)*F(q)*Fd(b)*F(j), keep_only_fully_contracted=True))

<IPython.core.display.Math object>

KroneckerDelta(_i, _q)*KroneckerDelta(_p, _q)*KroneckerDelta(a, b)*KroneckerDelta(i, j) + KroneckerDelta(_p, a)*KroneckerDelta(_q, b)*KroneckerDelta(i, j) - KroneckerDelta(_p, j)*KroneckerDelta(_q, i)*KroneckerDelta(a, b)

## Second Example: Two-body Hamiltonian 

First define Hamiltonian $H$ (not $H_N$)

Reference energy $E_0$ and normal ordering $H_N$

$E_0$ is obtained by fully contracting $H$.
and $H_N = H -E_0$.

In [12]:
# setup hamiltonian
p,q,r,s = symbols('p q r s',cls=Dummy)
f = AntiSymmetricTensor('f',(p,),(q,))
pr = Fd(p)*F(q)
v = AntiSymmetricTensor('v',(p,q),(r,s))
pqsr = Fd(p)*Fd(q)*F(s)*F(r)

#define the Hamiltonian
Hamiltonian=f*pr + Rational(1)/Rational(4)*v*pqsr
display(Math('H='+latex(Hamiltonian)))

#define indices for states above and below the Fermi level
index_rule = {
     'below':  'ijkl',
     'above':  'abcd',
     'general': 'pqrs'
     }
Hnormal = substitute_dummies(Hamiltonian,new_indices=True, pretty_indices=index_rule)
E0 = wicks(Hnormal,keep_only_fully_contracted=True)
display(Math('E_0='+latex(E0)))
#----define Hnormal 
Hnormal = Hnormal-E0
w = WildFunction('w')
Hnormal = collect(Hnormal, NO(w)) # why ? redundant? 
Hnormal = evaluate_deltas(Hnormal)
Hnormal 

<IPython.core.display.Math object>

<IPython.core.display.Math object>

-AntiSymmetricTensor(f, (_i,), (_i,)) + AntiSymmetricTensor(f, (_q,), (_p,))*CreateFermion(_q)*AnnihilateFermion(_p) - AntiSymmetricTensor(v, (_i, _i), (_i, _i))/4 - AntiSymmetricTensor(v, (_i, _i), (_i, _i))/4 + AntiSymmetricTensor(v, (_r, _s), (_p, _q))*CreateFermion(_s)*CreateFermion(_r)*AnnihilateFermion(_p)*AnnihilateFermion(_q)/4

One can get the same with Normal ordering operator. 

In [13]:
# setup hamiltonian
p,q,r,s = symbols('p q r s',cls=Dummy)
f = AntiSymmetricTensor('f',(p,),(q,))
pr = NO(Fd(p)*F(q))
v = AntiSymmetricTensor('v',(p,q),(r,s))
pqsr = NO(Fd(p)*Fd(q)*F(s)*F(r))

#define the Hamiltonian
Hnormal=f*pr + Rational(1)/Rational(4)*v*pqsr

Hnormal

AntiSymmetricTensor(f, (_p,), (_q,))*NO(CreateFermion(_p)*AnnihilateFermion(_q)) - AntiSymmetricTensor(v, (_p, _q), (_r, _s))*NO(CreateFermion(_p)*CreateFermion(_q)*AnnihilateFermion(_r)*AnnihilateFermion(_s))/4

 ## Compute Matrix element $<a b| H_N |c d >$

In [14]:
c,d = symbols('c, d',above_fermi=True)
a,b = symbols('a, b',above_fermi=True)

simplify(wicks(F(b)*F(a)*Hnormal*Fd(c)*Fd(d),keep_only_fully_contracted=True, simplify_kronecker_deltas=True))

KroneckerDelta(a, c)*AntiSymmetricTensor(f, (b,), (d,)) - KroneckerDelta(a, d)*AntiSymmetricTensor(f, (b,), (c,)) - KroneckerDelta(b, c)*AntiSymmetricTensor(f, (a,), (d,)) + KroneckerDelta(b, d)*AntiSymmetricTensor(f, (a,), (c,)) + AntiSymmetricTensor(v, (a, b), (c, d))

## compute 3-body matrix element $< a b c| V_N| d e f>$

In [15]:
# setup hamiltonian
v = AntiSymmetricTensor('v',(p,q),(r,s))
pqsr = NO(Fd(p)*Fd(q)*F(s)*F(r))
V_N = Rational(1)/Rational(4)*v*pqsr
a,b,c,d,e,f = symbols('a,b, c, d, e, f',above_fermi=True)

expression = wicks(F(c)*F(b)*F(a)*V_N*Fd(d)*Fd(e)*Fd(f),keep_only_fully_contracted=True, simplify_kronecker_deltas=True)
expression = evaluate_deltas(expression)
expression

KroneckerDelta(a, d)*AntiSymmetricTensor(v, (b, c), (e, f)) - KroneckerDelta(a, e)*AntiSymmetricTensor(v, (b, c), (d, f)) + KroneckerDelta(a, f)*AntiSymmetricTensor(v, (b, c), (d, e)) - KroneckerDelta(b, d)*AntiSymmetricTensor(v, (a, c), (e, f)) + KroneckerDelta(b, e)*AntiSymmetricTensor(v, (a, c), (d, f)) - KroneckerDelta(b, f)*AntiSymmetricTensor(v, (a, c), (d, e)) + KroneckerDelta(c, d)*AntiSymmetricTensor(v, (a, b), (e, f)) - KroneckerDelta(c, e)*AntiSymmetricTensor(v, (a, b), (d, f)) + KroneckerDelta(c, f)*AntiSymmetricTensor(v, (a, b), (d, e))

## Test CCSD equation terms 

Let us start from simplest one 

Define $T_1$ and $T_2$. (Use t for single and u for double amplitudes)

First term of Correlation Energy

In [16]:
#--this is wrong
t = AntiSymmetricTensor('t',(p,),(q,))
T1 = t*NO(Fd(p)*F(q))

u = AntiSymmetricTensor('u',(p,q),(r,s))
T2 = Rational(1)/Rational(4)*u*NO(Fd(p)*Fd(q)*F(s)*F(r))

wicks(Hnormal*T1-T1*Hnormal,keep_only_fully_contracted=True, simplify_kronecker_deltas=True)

0

Problem seems to be the declaration of symbols. Following give correct result

In [41]:
#---this seems to work 
from sympy import symbols, latex, WildFunction, collect, Rational, simplify, Dummy
from sympy.physics.secondquant import F, Fd, wicks, AntiSymmetricTensor, substitute_dummies, NO, evaluate_deltas
from IPython.display import display, Math

a,b,c,d,e,f = symbols('a,b,c,d,e,f',above_fermi=True, cls=Dummy)
i,j,k,l,m,n = symbols('i,j,k,l,m,n',below_fermi=True, cls=Dummy)
p,q,r,s,t,u = symbols('p,q,r,s,t,u', cls=Dummy)
#------- T1 and T2 amplitudes
g = AntiSymmetricTensor('g',(a,),(i,))
T1 = g*NO(Fd(a)*F(i))

h = AntiSymmetricTensor('h',(a,b),(i,j))
T2 = Rational(1,4)*h*NO(Fd(a)*Fd(b)*F(j)*F(i))
#------ Hamiltonian 
x = AntiSymmetricTensor('x',(p,),(q,))
H1 = x*NO(Fd(p)*F(q))
v = AntiSymmetricTensor('v',(p,q),(r,s))
H2 = Rational(1,4)*v*NO(Fd(p)*Fd(q)*F(s)*F(r))
Hnormal = H1+H2
#----- commutator 
wicks(Hnormal*T1-T1*Hnormal,keep_only_fully_contracted=True, simplify_kronecker_deltas=True)

AntiSymmetricTensor(g, (_a,), (_i,))*AntiSymmetricTensor(x, (_i,), (_a,))

What if one use the same symbol for amplitude and index?

t_ai = AntisymmetricTensor('t')

t = symbols('t',cls=Dummy) 

In this case, though it is displayed with the same symbol 't', 
program can distinguish them. 

# Following is from internet

In [42]:
"""
Calculates the Coupled-Cluster energy- and amplitude equations
See 'An Introduction to Coupled Cluster Theory' by
T. Daniel Crawford and Henry F. Schaefer III.
Other Resource : http://vergil.chemistry.gatech.edu/notes/sahan-cc-2010.pdf
"""

from sympy.physics.secondquant import (AntiSymmetricTensor, wicks,
        F, Fd, NO, evaluate_deltas, substitute_dummies, Commutator,
        simplify_index_permutations, PermutationOperator, KroneckerDelta )
from sympy import (
    symbols, Rational, latex, Dummy
)

pretty_dummies_dict = {
    'above': 'cdefgh',
    'below': 'klmno',
    'general': 'pqrstu'
}


def get_CC_operators():
    """
    Returns a tuple (T1,T2) of unique operators.
    """
    i = symbols('i', below_fermi=True, cls=Dummy)
    a = symbols('a', above_fermi=True, cls=Dummy)
    t_ai = AntiSymmetricTensor('t', (a,), (i,))
    ai = NO(Fd(a)*F(i))
    i, j = symbols('i,j', below_fermi=True, cls=Dummy)
    a, b = symbols('a,b', above_fermi=True, cls=Dummy)
    t_abij = AntiSymmetricTensor('t', (a, b), (i, j))
    abji = NO(Fd(a)*Fd(b)*F(j)*F(i))
   
    T1 = t_ai*ai
    T2 = Rational(1, 4)*t_abij*abji
    return (T1, T2)

def get_T3_operator():
    """
    return T3 
    """
    i,j,k = symbols('i,j,k', below_fermi=True, cls=Dummy)
    a,b,c = symbols('a,b,c', above_fermi=True, cls=Dummy)
    t_abcijk = AntiSymmetricTensor('t', (a, b, c), (i, j, k))
    abckji = NO(Fd(a)*Fd(b)*Fd(c)*F(k)*F(j)*F(i))
    T3 = Rational(1,6)*t_abcijk*abckji
    return T3 

def main():
    print()
    print("Calculates the Coupled-Cluster energy- and amplitude equations")
    print("See 'An Introduction to Coupled Cluster Theory' by")
    print("T. Daniel Crawford and Henry F. Schaefer III")
    print("Reference to a Lecture Series: http://vergil.chemistry.gatech.edu/notes/sahan-cc-2010.pdf")
    print()

    # setup hamiltonian
    p, q, r, s,t,u = symbols('p,q,r,s,t,u', cls=Dummy)
    f = AntiSymmetricTensor('f', (p,), (q,)) # f^p_q
    pr = NO(Fd(p)*F(q))    # a^+_p a_q 
    v = AntiSymmetricTensor('v', (p, q), (r, s)) # v^{pq}_{rs}
    pqsr = NO(Fd(p)*Fd(q)*F(s)*F(r)) # a^+_p a^+_q a_s a_r 
    w = AntiSymmetricTensor('w', (p, q, r), (s, t, u)) # w^{pqr}_{stu}
    pqruts = NO(Fd(p)*Fd(q)*Fd(r)*F(u)*F(t)*F(s))

    H = f*pr + Rational(1, 4)*v*pqsr 
    
    Hp = f*pr + Rational(1, 4)*v*pqsr + Rational(1,6)*w*pqruts # this is for test 
    
    print("Using the hamiltonian:", latex(H))

    print("Calculating 4 nested commutators")
    C = Commutator

    T1, T2 = get_CC_operators()
    T3 = get_T3_operator() 
    
    T = T1 + T2
    print("commutator 1...")
    comm1 = wicks(C(H, T))
    comm1 = evaluate_deltas(comm1)
    comm1 = substitute_dummies(comm1)

    T1, T2 = get_CC_operators()
    T = T1 + T2
    print("commutator 2...")
    comm2 = wicks(C(comm1, T))
    comm2 = evaluate_deltas(comm2)
    comm2 = substitute_dummies(comm2)

    T1, T2 = get_CC_operators()
    T = T1 + T2
    print("commutator 3...")
    comm3 = wicks(C(comm2, T))
    comm3 = evaluate_deltas(comm3)
    comm3 = substitute_dummies(comm3)

    T1, T2 = get_CC_operators()
    T = T1 + T2
    print("commutator 4...")
    comm4 = wicks(C(comm3, T))
    comm4 = evaluate_deltas(comm4)
    comm4 = substitute_dummies(comm4)

    print("construct Hausdorff expansion...")
    eq = H + comm1 + comm2/2 + comm3/6 + comm4/24
    eq = eq.expand()
    eq = evaluate_deltas(eq)
    eq = substitute_dummies(eq, new_indices=True,
            pretty_indices=pretty_dummies_dict)
    print("*********************")
    print()

    print("extracting CC equations from full Hbar")
    i, j, k, l = symbols('i,j,k,l', below_fermi=True)
    a, b, c, d = symbols('a,b,c,d', above_fermi=True)
    print()
    print("CC Energy:")
    en=wicks(eq, simplify_dummies=True, keep_only_fully_contracted=True)
    print(latex(en))
    print()
    print("CC T1:")
    eqT1 = wicks(NO(Fd(i)*F(a))*eq, simplify_kronecker_deltas=True, keep_only_fully_contracted=True)
    eqT1 = substitute_dummies(eqT1)
    print(latex(eqT1))
    print()
    print("CC T2:")
    eqT2 = wicks(NO(Fd(i)*Fd(j)*F(b)*F(a))*eq, simplify_dummies=True, keep_only_fully_contracted=True, simplify_kronecker_deltas=True)
    P = PermutationOperator
    eqT2 = simplify_index_permutations(eqT2, [P(a, b), P(i, j)])
    print(latex(eqT2))

    return en, eqT1, eqT2

#===Run===============
en, eqT1, eqT2 = main()


Calculates the Coupled-Cluster energy- and amplitude equations
See 'An Introduction to Coupled Cluster Theory' by
T. Daniel Crawford and Henry F. Schaefer III
Reference to a Lecture Series: http://vergil.chemistry.gatech.edu/notes/sahan-cc-2010.pdf

Using the hamiltonian: f^{p}_{q} \left\{a^\dagger_{p} a_{q}\right\} - \frac{v^{pq}_{rs} \left\{a^\dagger_{p} a^\dagger_{q} a_{r} a_{s}\right\}}{4}
Calculating 4 nested commutators
commutator 1...
commutator 2...
commutator 3...
commutator 4...
construct Hausdorff expansion...
*********************

extracting CC equations from full Hbar

CC Energy:
f^{k}_{c} t^{c}_{k} - \frac{t^{c}_{l} t^{d}_{k} v^{kl}_{cd}}{2} + \frac{t^{cd}_{kl} v^{kl}_{cd}}{4}

CC T1:
- f^{k}_{c} t^{c}_{i} t^{a}_{k} + f^{k}_{c} t^{ac}_{ik} - f^{k}_{i} t^{a}_{k} + f^{a}_{c} t^{c}_{i} + f^{a}_{i} - t^{c}_{k} t^{d}_{i} t^{a}_{l} v^{kl}_{cd} - t^{c}_{k} t^{d}_{i} v^{ak}_{cd} + t^{c}_{k} t^{a}_{l} v^{kl}_{ic} + t^{c}_{k} t^{ad}_{il} v^{kl}_{cd} + t^{c}_{k} v^{ak}_{ic} - \fra

In [43]:
en

AntiSymmetricTensor(f, (_k,), (_c,))*AntiSymmetricTensor(t, (_c,), (_k,)) - AntiSymmetricTensor(t, (_c,), (_l,))*AntiSymmetricTensor(t, (_d,), (_k,))*AntiSymmetricTensor(v, (_k, _l), (_c, _d))/2 + AntiSymmetricTensor(t, (_c, _d), (_k, _l))*AntiSymmetricTensor(v, (_k, _l), (_c, _d))/4

In [44]:
eqT1

-AntiSymmetricTensor(f, (_k,), (_c,))*AntiSymmetricTensor(t, (_c,), (i,))*AntiSymmetricTensor(t, (a,), (_k,)) + AntiSymmetricTensor(f, (_k,), (_c,))*AntiSymmetricTensor(t, (a, _c), (i, _k)) - AntiSymmetricTensor(f, (_k,), (i,))*AntiSymmetricTensor(t, (a,), (_k,)) + AntiSymmetricTensor(f, (a,), (_c,))*AntiSymmetricTensor(t, (_c,), (i,)) + AntiSymmetricTensor(f, (a,), (i,)) - AntiSymmetricTensor(t, (_c,), (_k,))*AntiSymmetricTensor(t, (_d,), (i,))*AntiSymmetricTensor(t, (a,), (_l,))*AntiSymmetricTensor(v, (_k, _l), (_c, _d)) - AntiSymmetricTensor(t, (_c,), (_k,))*AntiSymmetricTensor(t, (_d,), (i,))*AntiSymmetricTensor(v, (a, _k), (_c, _d)) + AntiSymmetricTensor(t, (_c,), (_k,))*AntiSymmetricTensor(t, (a,), (_l,))*AntiSymmetricTensor(v, (_k, _l), (i, _c)) + AntiSymmetricTensor(t, (_c,), (_k,))*AntiSymmetricTensor(t, (a, _d), (i, _l))*AntiSymmetricTensor(v, (_k, _l), (_c, _d)) + AntiSymmetricTensor(t, (_c,), (_k,))*AntiSymmetricTensor(v, (a, _k), (i, _c)) - AntiSymmetricTensor(t, (_c,), (i

In [45]:
eqT2

AntiSymmetricTensor(f, (_k,), (_c,))*AntiSymmetricTensor(t, (_c,), (i,))*AntiSymmetricTensor(t, (a, b), (j, _k))*PermutationOperator(i, j) + AntiSymmetricTensor(f, (_k,), (_c,))*AntiSymmetricTensor(t, (a,), (_k,))*AntiSymmetricTensor(t, (b, _c), (i, j))*PermutationOperator(a, b) + AntiSymmetricTensor(f, (_k,), (i,))*AntiSymmetricTensor(t, (a, b), (j, _k))*PermutationOperator(i, j) - AntiSymmetricTensor(f, (a,), (_c,))*AntiSymmetricTensor(t, (b, _c), (i, j))*PermutationOperator(a, b) + AntiSymmetricTensor(t, (_c,), (_k,))*AntiSymmetricTensor(t, (_d,), (i,))*AntiSymmetricTensor(t, (a, b), (j, _l))*AntiSymmetricTensor(v, (_k, _l), (_c, _d))*PermutationOperator(i, j) + AntiSymmetricTensor(t, (_c,), (_k,))*AntiSymmetricTensor(t, (a,), (_l,))*AntiSymmetricTensor(t, (b, _d), (i, j))*AntiSymmetricTensor(v, (_k, _l), (_c, _d))*PermutationOperator(a, b) - AntiSymmetricTensor(t, (_c,), (_k,))*AntiSymmetricTensor(t, (a, _d), (i, j))*AntiSymmetricTensor(v, (b, _k), (_c, _d))*PermutationOperator(a, 

## Let us look at each steps

It looks like if one can reduce the antisymmetric ternsors with the same index to zero,
the calculation may be faster. But, how to implement it?? 

In [46]:

from sympy.physics.secondquant import (AntiSymmetricTensor, wicks,
        F, Fd, NO, evaluate_deltas, substitute_dummies, Commutator,
        simplify_index_permutations, PermutationOperator, KroneckerDelta )
from sympy import (
    symbols, Rational, latex, Dummy
)

pretty_dummies_dict = {
    'above': 'cdefgh',
    'below': 'klmno',
    'general': 'pqrstu'
}
# setup hamiltonian
p, q, r, s,t,u = symbols('p,q,r,s,t,u', cls=Dummy)
f = AntiSymmetricTensor('f', (p,), (q,)) # f^p_q
pr = NO(Fd(p)*F(q))    # a^+_p a_q 
v = AntiSymmetricTensor('v', (p, q), (r, s)) # v^{pq}_{rs}
pqsr = NO(Fd(p)*Fd(q)*F(s)*F(r)) # a^+_p a^+_q a_s a_r 
w = AntiSymmetricTensor('w', (p, q, r), (s, t, u)) # w^{pqr}_{stu}
pqruts = NO(Fd(p)*Fd(q)*Fd(r)*F(u)*F(t)*F(s))

H = f*pr + Rational(1, 4)*v*pqsr 
C = Commutator

T1, T2 = get_CC_operators()
T3 = get_T3_operator() 
    
T = T1 + T2
print("commutator 1...")
comm1 = wicks(C(H, T))
comm1 = evaluate_deltas(comm1)
comm1 = substitute_dummies(comm1)

Using the hamiltonian: f^{p}_{q} \left\{a^\dagger_{p} a_{q}\right\} - \frac{v^{pq}_{rs} \left\{a^\dagger_{p} a^\dagger_{q} a_{r} a_{s}\right\}}{4}
Calculating 4 nested commutators
commutator 1...


C(H,T) gives [H,T] as output. not expanded.
wicks(C(H,T)) make contractions replace 

In [52]:
C(H,T)

-AntiSymmetricTensor(f, (_p,), (_q,))*AntiSymmetricTensor(t, (_a,), (_i,))*Commutator(NO(CreateFermion(_a)*AnnihilateFermion(_i)), NO(CreateFermion(_p)*AnnihilateFermion(_q))) - AntiSymmetricTensor(f, (_p,), (_q,))*AntiSymmetricTensor(t, (_a, _b), (_i, _j))*Commutator(NO(CreateFermion(_p)*AnnihilateFermion(_q)), NO(CreateFermion(_a)*CreateFermion(_b)*AnnihilateFermion(_i)*AnnihilateFermion(_j)))/4 + AntiSymmetricTensor(t, (_a,), (_i,))*AntiSymmetricTensor(v, (_p, _q), (_r, _s))*Commutator(NO(CreateFermion(_a)*AnnihilateFermion(_i)), NO(CreateFermion(_p)*CreateFermion(_q)*AnnihilateFermion(_r)*AnnihilateFermion(_s)))/4 - AntiSymmetricTensor(t, (_a, _b), (_i, _j))*AntiSymmetricTensor(v, (_p, _q), (_r, _s))*Commutator(NO(CreateFermion(_a)*CreateFermion(_b)*AnnihilateFermion(_i)*AnnihilateFermion(_j)), NO(CreateFermion(_p)*CreateFermion(_q)*AnnihilateFermion(_r)*AnnihilateFermion(_s)))/16

In [53]:
wicks(C(H,T))

KroneckerDelta(_a, _a)*KroneckerDelta(_a, _p)*KroneckerDelta(_a, _q)*AntiSymmetricTensor(f, (_p,), (_q,))*AntiSymmetricTensor(t, (_a,), (_i,))*NO(CreateFermion(_a)*AnnihilateFermion(_i)) + KroneckerDelta(_a, _a)*KroneckerDelta(_a, _q)*KroneckerDelta(_i, _i)*KroneckerDelta(_i, _p)*AntiSymmetricTensor(f, (_p,), (_q,))*AntiSymmetricTensor(t, (_a,), (_i,)) - KroneckerDelta(_a, _a)*KroneckerDelta(_a, _q)*KroneckerDelta(_i, _p)*AntiSymmetricTensor(f, (_p,), (_q,))*AntiSymmetricTensor(t, (_a,), (_i,))*NO(AnnihilateFermion(_i)*CreateFermion(_i)) + KroneckerDelta(_a, _a)*KroneckerDelta(_a, _p)*KroneckerDelta(_a, _q)*KroneckerDelta(_a, _r)*KroneckerDelta(_a, _s)*AntiSymmetricTensor(t, (_a,), (_i,))*AntiSymmetricTensor(v, (_p, _q), (_r, _s))*NO(CreateFermion(_a)*CreateFermion(_a)*AnnihilateFermion(_i)*AnnihilateFermion(_a))/4 - KroneckerDelta(_a, _a)*KroneckerDelta(_a, _p)*KroneckerDelta(_a, _q)*KroneckerDelta(_a, _r)*KroneckerDelta(_i, _s)*AntiSymmetricTensor(t, (_a,), (_i,))*AntiSymmetricTensor